# Object detectie

> Het doel van deze opdracht is om een object te grijpen met een robot, gebruik makende van een camera. Een eerste stap is om dit object te detecteren met een camera. We nemen hiervoor een afbeelding en bewerken het beeld tot we het pixelcoördinaat (u, v) van het center van het object hebben. In dit script zal je dus een afbeelding van het object inladen, image processing toepassen, en tot slot de pixelcoördinaat bepalen.

# 1. Importeer bibliotheken

> Deze bibliotheken zijn nodig voor het volbrengen van dit script. In de CameraCalibration bibliotheek (die je vindt in de Classes folder op je Google Drive) vind je de code achter de commando's die je hier zal gebruiken. Deze cel zal een error geven als je niet met je Google Drive bent verbonden. Zorg er dus voor dat je links in de bestanden een folder 'drive' ziet staan naast de default 'sample_data'-folder. Indien niet, verbind met de drive door links op het mapje te klikken en vervolgens op het Google Drive symbool te klikken. Druk vervolgens nog eens op 'Runtime', 'Runtime opnieuw starten', om te refreshen.

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/object_pose_estimation_online')
from Classes.CameraCalibration import *
from google.colab.patches import cv2_imshow


# 2. Importeer het beeld voor object detectie

> De afbeelding van het object in het robotveld is opgeslaan onder de map /content/drive/My Drive/object_pose_estimation_online/data/object_detection_images. De functie 'get_image_path_names' geeft je alle padnamen naar de afbeeldingen in die folder. Als je de output uitprint kan je zien dat er een padnaam aanwezig is naar een afbeelding.


In [ ]:
# Get all image's path names
object_detection_images_file = '/content/drive/My Drive/object_pose_estimation_online/data/object_detection_images/*.png'
images_path_names = get_image_path_names(object_detection_images_file)
print(images_path_names)

> We kunnen deze afbeelding inladen met de 'cv2.imread' functie en kijken hoe deze afbeelding eruit ziet met de 'cv2_imshow' functie.  

In [ ]:
# Preview image
image_path = images_path_names[0]
image = cv2.imread(image_path)
cv2_imshow(image)

# 3. Image processing


> Al eerste stap, zetten we het beeld om naar grayscale om de grote hoeveelheid data te reduceren. Kleur informatie is in deze toepassing namelijk niet van belang.

In [ ]:
# To grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray_image)



> Vervolgens filteren we het beeld met een 7x7 Gaussiaanse filter om de hoogfrequente ruis eruit te krijgen. Dit zal ons helpen bij het detecteren van 'edges' in het beeld.



In [ ]:
# Gaussian blur
blurred_image = cv2.GaussianBlur(gray_image, (7, 7), 0)
cv2_imshow(blurred_image)



> Hier komt de stap waarbij we in de geblurde afbeelding randen zullen detecteren. Het gebruikte 'Canny Edge' algoritme zal pixels detecteren met een hoge intensiteitsgradiënt (groot verschil in pixelwaardes tussen twee opeenvolgende pixels). Als output krijgen we een binair (enkel 0 en 255 waarden) beeld met alle randen.



In [ ]:
# Canny Edge detection
canny = cv2.Canny(blurred_image, 50, 200)
cv2_imshow(canny)



> Je kan zien dat er links en rechts wat dingen mee gedetecteerd worden die we niet in ons resultaat willen. We zullen het beeld dus knippen. We definiëren met andere woorden een 'Region of Interest' (ROI) met enkel de ruimte die voor ons interessant is, namelijk het robot veld. We kijken eerst naar het aantal pixels in u- en v- richting dat de afbeelding heeft, en daarop baseren we ons om de pixels buiten de ROI op 0 te zetten (zwart).



In [ ]:
# Shape of image in u- and v- pixels
print("Image shape: " + str(canny.shape))

In [ ]:
# Define ROI
canny[:, 0:450] = 0  # All pixels from u-value 0 till 450 are put to zero for all v
canny[:, 900:1280] = 0  # All pixels from u-value 900 till 1280 are put to zero for all v
roi = canny
cv2_imshow(roi)



> Als volgt gaan we de gedetecteerde randen die slechts 1 pixel breed zijn 'dilaten' (zoek op) om de eventuele gaten die in de rand zouden voorkomen te dichten.



In [ ]:
# Dilate image
roi_d = cv2.dilate(roi, None, iterations=2)
cv2_imshow(roi_d)



> Vervolgens 'eroderen' (zoek op) we de afbeelding terug om de rand weer te verkleinen. Maar de gaten blijven zo wel opgevuld zodat we zeker zijn van een continue rand.



In [ ]:
# Erode image
roi_e = cv2.erode(roi_d, None, iterations=2)
cv2_imshow(roi_e)

# 4. Contour detection

Nu we de image processing hebben beïndigd kunnen we overgaan tot het detecteren van 'contouren' (zoek op) in de afbeelding. Deze kunnen worden gevisualiseerd op de afbeelding.

In [ ]:
# Find contours
contours = cv2.findContours(roi_e, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
image_with_contours = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 3)
cv2_imshow(image_with_contours)



> We kunnen rond deze contour een bounding box definiëren.



In [ ]:
# Compute the bounding box of the contour
box = cv2.minAreaRect(contours[0])
box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
image_with_box = cv2.drawContours(image.copy(), [box.astype("int")], -1, (0, 255, 0), 2)
cv2_imshow(image_with_box)



> Als we van de vier hoekpunten van deze bounding box gedefiniëerd in pixelcoördinaten de gemiddelde coördinaat nemen, dan bekomen we het centerpunt van ons object in pixelcoördinaten.



In [ ]:
# Print box coördinates
print("Coördinaten van de bounding box in pixel coördinaten (u,v): \n\n" + str(box))
mean_coordinate = np.mean(box, axis=0)
print("\nCenterpunt van de box in pixel coördinaten (u,v): \n\n" + str(mean_coordinate))



> We kunnen dit visualiseren door een punt te projecteren op de afbeelding met deze pixel coördinaten. Dit punt zullen we nu projecteren in baselink frame.



In [ ]:
cv2.circle(image_with_box, (int(mean_coordinate[0]), int(mean_coordinate[1])), 3, [255, 0, 0], thickness=3)
cv2_imshow(image_with_box)

> We hebben dus nu het pixelcoördinaat van het center van het object. Helaas is de robot hier niet zoveel mee en moeten we dit nog omzetten naar coördinaten die voor de robot nuttig zijn. We slaan hieronder het pixelcoordinaat nog op om te kunnen gebruiken in volgend script.

In [ ]:
# Save pixel coordinate
obj_pixel_coor = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/obj_pixel_coor'
save_to_numpy(obj_pixel_coor, mean_coordinate)

# Vragen bij dit script:

*  VRAAG 1: Wat is 'dilaten'? En wat is het nut ervan?
*  VRAAG 2: Wat is 'eroderen'? En wat is het nut ervan?
*  VRAAG 3: Wat is een 'contour'? En wat is het nut ervan?

